# Перцептрон для восприятия цифр
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW05] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать классический перцептрон, немного улчшить его, и протестировать результат на классической задаче определния цифр по изображениям.

In [7]:
import numpy as np
from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import datasets
import copy

### Задание 1 (3 балла)
Для начала реализуем простой перцептрон.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

`fit(X, y)` - инициализирует веса перцептрона, а затем обновляет их в течении `iterations` итераций. 

#### Параметры конструктора
`iterations` - количество итераций обучения перцептрона

#### Поля
`w` - веса перцептрона размерности `X.shape[1] + 1`. `w[0]` должен соответстовать константе, `w[1:]` - коэффициентам компонент элемента `X`.

In [8]:
class Perceptron:
    def __init__(self, iterations=100):
        self.iterations = iterations
        self.X = None
        self.y = None
        self.w = None
        self.maxs = None
        self.mins = None

    def iteration(self):
        wrongs = [i for i in range(len(self.X)) if self.y[i] * np.dot(self.w, self.X[i]) < 0]
        index = np.random.choice(wrongs)
        self.w = self.w + self.y[index] * self.X[index]

    def normalise(self, x):
        return np.hstack(([1], (x - self.mins) / (self.maxs - self.mins)))

    def fit(self, X, y):
        self.maxs = np.array([np.max(X[:, j]) for j in range(X.shape[1])])
        self.mins = np.array([np.min(X[:, j]) for j in range(X.shape[1])])
        self.X = np.array(
            [self.normalise(x) for x in X])
        self.y = (y * 2) - 1
        self.w = np.random.random((X.shape[1] + 1))
        self.bestw = self.w
        self.besterror = len(X)
        for i in range(self.iterations):
            self.iteration()

    def predict(self, X):
        return (1 + np.array([np.sign(np.dot(self.w, self.normalise(x))) for x in X])) / 2

    def getw(self):
        w1 = self.w[1:] / (self.maxs - self.mins)
        return np.hstack(([self.w[0] - np.dot(w1, self.mins)], w1))


Посмотрим на то, как наш перцептрон справляется с различными наборами данных

In [9]:
X, true_labels = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5]])
c = Perceptron()
c.fit(X, true_labels)
visualize(X, true_labels, np.array(c.predict(X)), c.getw())

NameError: name 'visualize' is not defined

In [ ]:
X, true_labels = make_moons(400, noise=0.075)
c = Perceptron()
c.fit(X, true_labels)
visualize(X, true_labels, np.array(c.predict(X)), c.getw())

### Задание 2 (2 балл)
Проблема обычной реализации перцептрона в том, что закончить его обучение мы можем с неоптимальными весами, т.к. точность разбиения в зависимости от числа итераций не растет монотонно. Чтобы этого избежать, мы можем оставлять в конце веса той итерации, на которой мы лучше всего разбивали множество `X`.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

`fit(X, y)` - инициализирует веса перцептрона, а затем обновляет их в течении `iterations` итераций. В конце обучения оставляет лучшие веса. 

#### Параметры конструктора
`iterations` - количество итераций обучения перцептрона

#### Поля
`w` - веса перцептрона размерности `X.shape[1] + 1`. `w[0]` должен соответстовать константе, `w[1:]` - коэффициентам компонент элемента `X`.

In [ ]:
class PerceptronBest:
    def __init__(self, iterations=100):
        self.iterations = iterations
        self.X = None
        self.y = None
        self.w = None
        self.maxs = None
        self.mins = None
        self.besterror = None
        self.bestw = None

    def iteration(self):
        wrongs = [i for i in range(len(self.X)) if self.y[i] * np.dot(self.w, self.X[i]) < 0]
        if len(wrongs) < self.besterror:
            self.besterror = len(wrongs)
            self.bestw = self.w
        index = np.random.choice(wrongs)
        self.w = self.w + self.y[index] * self.X[index]

    def normalise(self, x):
        return np.hstack(([1], (x - self.mins) / (self.maxs - self.mins)))

    def fit(self, X, y):
        self.maxs = np.array([np.max(X[:, j]) for j in range(X.shape[1])])
        self.mins = np.array([np.min(X[:, j]) for j in range(X.shape[1])])
        self.X = np.array(
            [self.normalise(x) for x in X])
        self.y = (y * 2) - 1
        self.w = np.random.random((X.shape[1] + 1))
        self.bestw = self.w
        self.besterror = len(X)
        for i in range(self.iterations):
            self.iteration()
        self.w = self.bestw

    def predict(self, X):
        return (1 + np.array([np.sign(np.dot(self.w, self.normalise(x))) for x in X])) / 2

    def getw(self):
        w1 = self.w[1:] / (self.maxs - self.mins)
        return np.hstack(([self.w[0] - np.dot(w1, self.mins)], w1))


In [ ]:
def visualize(X, labels_true, labels_pred, w):
    unique_labels = np.unique(labels_true)
    unique_colors = dict([(l, c) for l, c in zip(unique_labels, [[0.8, 0., 0.], [0., 0., 0.8]])])
    plt.figure(figsize=(9, 9))

    if w[1] == 0:
        plt.plot([X[:, 0].min(), X[:, 0].max()], w[0] / w[2])
    elif w[2] == 0:
        plt.plot(w[0] / w[1], [X[:, 1].min(), X[:, 1].max()])  
    else:
        mins, maxs = X.min(axis=0), X.max(axis=0)
        pts = [[mins[0], -mins[0] * w[1] / w[2] - w[0] / w[2]],
               [maxs[0], -maxs[0] * w[1] / w[2] - w[0] / w[2]],
               [-mins[1] * w[2] / w[1] - w[0] / w[1], mins[1]],
               [-maxs[1] * w[2] / w[1] - w[0] / w[1], maxs[1]]]
        pts = [(x, y) for x, y in pts if mins[0] <= x <= maxs[0] and mins[1] <= y <= maxs[1]]
        x, y = list(zip(*pts))
        plt.plot(x, y, c=(0.75, 0.75, 0.75), linestyle="--")
    
    colors_inner = [unique_colors[l] for l in labels_true]
    colors_outer = [unique_colors[l] for l in labels_pred]
    plt.scatter(X[:, 0], X[:, 1], c=colors_inner, edgecolors=colors_outer)
    plt.show()

Посмотрим на то, как наш перцептрон справляется с различными наборами данных

In [ ]:
X, true_labels = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5]])
c = PerceptronBest()
c.fit(X, true_labels)
visualize(X, true_labels, np.array(c.predict(X)), c.getw())

In [ ]:
X, true_labels = make_moons(400, noise=0.075)
c = PerceptronBest()
c.fit(X, true_labels)
visualize(X, true_labels, np.array(c.predict(X)), c.getw())

### Задание 3 (1 балл)
Реализуйте метод `transform_images(images)`, который преобразует изображения в двумерные векторы. Значение компонент векторов придумайте сами и напишите в виде комментария к методу.

In [ ]:
#считает среднее p(1-p) где p = доля единиц в столбце. Центральные столбцы учитываются с большим весом
def first_feature(image):
    rounded = np.array([[round(p) for p in v] for v in image])
    count = np.array([rounded[:, j].tolist().count(1) for j in range(8)]) / 8
    weights = [0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.0]
    return np.dot(weights, 4 * count * (1 - count))

#считает для всех точек = 0 колво единиц над точкой * колво единиц под точкой
def second_feature(image):
    sum = 0
    rounded = np.array([[round(p) for p in v] for v in image])
    for j in range(8):
        for i in range(8):
            s1 = np.sum(rounded[:i, j])
            s2 = np.sum(rounded[i + 1:, j])
            sum += (1 - rounded[i, j]) * s1 * s2
    return sum


def transform_images(images):
    return np.array([[first_feature(x), second_feature(x)] for x in images])

def get_digits(y0=1, y1=5):
    data = datasets.load_digits()
    images, labels = data.images, data.target
    mask = np.logical_or(labels == y0, labels == y1)
    labels = labels[mask]
    images = images[mask]
    images /= np.max(images)
    labels = np.array([(x - y0) // (y1-y0) for x in labels])
    X = transform_images(images)
    return X, labels

### Задание 4 (4 балла)
Теперь посмотрим на точность обычного перцептрона и перцептрона выбором лучшей итерации. Для тестирования будем использовать цифры 1 и 5. Необходимо использовать реализованное выше преобразование, т.е. только векторы из 2х признаков. 

Оценка за это задание выставляется следующим образом:
1. 1 балл - что-то обучается, картинки рисуются
2. 2 балла - точность хотя бы одного из классификаторов на тестовой выборке больше 80%
3. 4 балла - точность хотя бы одного из классификаторов на тестовой выборке больше 90%

__Обратите внимание, что выборка разбивается детерминировано, т.е. неи смысла считать среднюю точность__ 

In [ ]:
X, y = get_digits()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)

In [ ]:
c = Perceptron(iterations=100000)
c.fit(X_train, y_train)
visualize(X_train, y_train, np.array(c.predict(X_train)), c.getw())
print("Accuracy:", np.mean(c.predict(X_test) == y_test))

In [ ]:
c = PerceptronBest(iterations=100000)
c.fit(X_train, y_train)
visualize(X_train, y_train, np.array(c.predict(X_train)), c.getw())
print("Accuracy:", np.mean(c.predict(X_test) == y_test))

А теперь посчитаем среднюю точность по всем возможным парам цифр

In [ ]:
accs = []
for y0, y1 in [(y0, y1) for y0 in range(9) for y1 in range(y0+1, 10)]:
    X, y = get_digits(y0, y1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
    c = Perceptron(iterations=20000)
    c.fit(X_train, y_train)
    accs.append(np.mean(c.predict(X_test) == y_test))
print("Mean accuracy:", np.mean(accs))

In [ ]:
accs = []
for y0, y1 in [(y0, y1) for y0 in range(9) for y1 in range(y0+1, 10)]:
    X, y = get_digits(y0, y1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
    c = PerceptronBest(iterations=20000)
    c.fit(X_train, y_train)
    accs.append(np.mean(c.predict(X_test) == y_test))
print("Mean accuracy:", np.mean(accs))